# Layers

The most general layers in `kgcnn` take normal and ragged tensor as input. The graph oriented operations are im


1. The most general layers that kept maintained beyond different models with proper documentation are located in `kgcnn.layers`. These are:
    * `activ` Activation layers with learnable parameters.
    * `aggr` Aggregatoion layers for e.g. aggregating edge messages. 
    * `attention` Layers for graph attention.
    * `casting` Layers for casting tensor formats.
    * `conv` Basic convolution layers.
    * `gather` Layers around tf.gather.
    * `geom` Geometry operations.
    * `message` Message passing base layer.
    * `mlp` Multi-layer perceptron for graphs.
    * `modules` Keras layers and modules to support ragged tensor input.
    * `norm` Normalization layers for graph tensors.
    * `polynom` Layers for Polynomials.
    * `pooling` General layers for standard aggregation and pooling.
    * `relational` Relational message processing.
    * `scale` Scaling layer to (constantly) rescale e.g. graph output.
    * `set2set` Set2Set type architectures for e.g. pooling nodes.
    * `update` Some node/edge update layers.


> **NOTE**: Please check https://kgcnn.readthedocs.io/en/latest/kgcnn.layers.html for documentation of each layer.

## Implementaion details

TODO

> **NOTE**: You can find this page as jupyter notebook in https://github.com/aimat-lab/gcnn_keras/tree/master/docs/source